# Data preparation for the learning phase

Install of the library which contains all data

In [ ]:
pip install fastf1

In [ ]:
import fastf1

Getting all data for most recent years with same compounds types

In [ ]:
schedules = []
for year in [2019, 2020, 2021, 2022, 2023]:
    schedules.append(fastf1.get_event_schedule(year))

## Examples of use
Example of preparation phase on a single race

In [ ]:
race = schedules[0].get_event_by_round(1).get_race()
race.load()
race.laps

In [ ]:
race.laps.columns

In [ ]:
race.laps.loc[:, [False, True, True, True, True, True,
       False, False, False, False, False,
       False, False, False,
       False, False, False, False, False,
       True, True, True, True, False,
       False, False, False, False, False,
       False, False]]

In [ ]:
race.results.columns

In [ ]:
race.results.loc[:, [True, False, True, False, True,
       False, False, False, False, False,
       False, False, True, False,
       True, False, False, False, False, False, False]]

In [ ]:
race.weather_data.columns

In [ ]:
race.weather_data.loc[:, [False, True, True, True, True, True,
       False, True]]

## Extraction

Preparing list of colums which have to be selected in the tables of interest:
- Laps table: Driver, DriverNumber, LapTime, LapNumber, Stint, Compound, TyreLife, FreshTyre, Team
- Results table: DriverNumber, Abbreviation, TeamName, Position, GridPosition
- Weather Data table: AirTemp, Humidity, Pressure, Rainfall, TrackTemp, WindSpeed

In [ ]:
laps_list = [False, True, True, True, True, True,
       False, False, False, False, False,
       False, False, False,
       False, False, False, False, False,
       True, True, True, True, False,
       False, False, False, False, False,
       False, False]

results_list = [True, False, True, False, True,
       False, False, False, False, False,
       False, False, True, False,
       True, False, False, False, False, False, False]

weather_list = [False, True, True, True, True, True,
       False, True]

Loading of all the data and extracting only column of interest

In [ ]:
peryear_races = []
peryear_races_results = []
peryear_races_laps = []
peryear_races_weather = []

i = 0
for schedule in schedules:
    peryear_races.append({})
    peryear_races_results.append({})
    peryear_races_laps.append({})
    peryear_races_weather.append({})
    
    for round in range(1, len(schedule)-1):
        peryear_races[i][round] = schedule.get_event_by_round(round).get_race()
        peryear_races[i][round].load()
        peryear_races_results[i][round] = peryear_races[i][round].results.loc[:, results_list]
        peryear_races_laps[i][round] = peryear_races[i][round].laps.loc[:, laps_list]
        peryear_races_weather[i][round] = peryear_races[i][round].weather_data.loc[:, weather_list]
    
    i+=1
    